In [1]:
# Importando as bibliotecas

import pandas as pd
import plotly.express as px
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage

In [2]:
# Importando a base de dados

tabela_vendas = pd.read_excel('/content/Vendas_Base_de_Dados.xlsx')
display(tabela_vendas)

,Cód.,Data,Loja,Produto,Quantidade,Valor Unitário
0,1,5/18/2022,Maranhão,Caixa de som,4,79
1,2,7/24/2022,Santa Catarina,Notebook,9,3789
2,3,12/20/2022,Rio de Janeiro,Notebook,3,3789
3,4,1/22/2022,Espírito Santo,Nobreak,9,1279
4,5,9/11/2022,Tocantins,Impressora,9,699
...,...,...,...,...,...,...
14995,14996,8/18/2022,São Paulo,Webcam,4,150
14996,14997,1/6/2022,São Paulo,Impressora,9,699
14997,14998,11/13/2022,São Paulo,Notebook,9,3789
14998,14999,6/29/2022,São Paulo,Caixa de som,6,79


In [3]:
# Calcular o produto mais vendido (em quantidade)

tb_quantidade_produto = tabela_vendas.groupby('Produto').sum(numeric_only=True)
tb_quantidade_produto = tb_quantidade_produto[['Quantidade']]
tb_quantidade_produto = tb_quantidade_produto.sort_values(by='Quantidade', ascending=False)
display(tb_quantidade_produto)

,Quantidade
Produto,
Impressora,13285
Notebook,12930
Monitor,9923
Nobreak,6778
Tablet,6684
Teclado,6631
Webcam,6562
Caixa de som,6535
PC,6518


In [4]:
# Calcular o produto mais vendido (Em Faturamento)

tabela_vendas['Faturamento'] = tabela_vendas['Quantidade'] * tabela_vendas['Valor Unitário']
tb_faturamento_produto = tabela_vendas.groupby('Produto').sum(numeric_only=True)
tb_faturamento_produto = tb_faturamento_produto[['Faturamento']].sort_values(by='Faturamento', ascending=False)
display(tb_faturamento_produto)

,Faturamento
Produto,
Notebook,48991770
PC,38456200
Monitor,11907600
Impressora,9286215
Tablet,9016716
Nobreak,8669062
Teclado,1193580
Webcam,984300
Mouse,725872


In [5]:
# Calcular a loja ou estado que mais vendeu (em Faturamento)
tb_faturamento_loja = tabela_vendas.groupby('Loja').sum(numeric_only=True)
tb_faturamento_loja = tb_faturamento_loja[['Faturamento']].sort_values(by='Faturamento', ascending=False)
display(tb_faturamento_loja.head())

,Faturamento
Loja,
São Paulo,41018776
Rio de Janeiro,12274351
Minas Gerais,11491527
Paraná,7611645
Pernambuco,7041123


In [6]:
# Calcular o ticket médio por Loja/Estado

tabela_vendas['Ticket_medio'] = tabela_vendas['Valor Unitário']
tb_ticket_medio = tabela_vendas.groupby('Loja').mean(numeric_only=True)
tb_ticket_medio = tb_ticket_medio[['Ticket_medio']].sort_values(by='Ticket_medio', ascending=False)
display(tb_ticket_medio.head())

,Ticket_medio
Loja,
Acre,2142.769231
Sergipe,1735.333333
Tocantins,1734.306452
Rio Grande do Sul,1690.043909
Pará,1683.746177


In [7]:
# Plotando um gráfico loja que mais vendeu (em Faturamento)

grafico = px.bar(tb_faturamento_loja, y='Faturamento', x=tb_faturamento_loja.index)
grafico.show()

In [8]:
from babel.numbers import format_currency

tb_faturamento_loja_formatado = pd.DataFrame(tb_faturamento_loja['Faturamento'].apply(lambda x: format_currency(x, 'BRL', locale='pt_BR')))
tb_faturamento_loja_formatado = tb_faturamento_loja_formatado.reset_index()

# Formatação Html
tb_faturamento_loja_formatado = tb_faturamento_loja_formatado.to_html(index=False, justify='center', border=0).replace('<tbody>', '<tbody style="text_align: center; color: #484848; background: #F7F7F7">')

display(tb_faturamento_loja_formatado)

'<table class="dataframe">\n  <thead>\n    <tr style="text-align: center;">\n      <th>Loja</th>\n      <th>Faturamento</th>\n    </tr>\n  </thead>\n  <tbody style="text_align: center; color: #484848; background: #F7F7F7">\n    <tr>\n      <td>São Paulo</td>\n      <td>R$\xa041.018.776,00</td>\n    </tr>\n    <tr>\n      <td>Rio de Janeiro</td>\n      <td>R$\xa012.274.351,00</td>\n    </tr>\n    <tr>\n      <td>Minas Gerais</td>\n      <td>R$\xa011.491.527,00</td>\n    </tr>\n    <tr>\n      <td>Paraná</td>\n      <td>R$\xa07.611.645,00</td>\n    </tr>\n    <tr>\n      <td>Pernambuco</td>\n      <td>R$\xa07.041.123,00</td>\n    </tr>\n    <tr>\n      <td>Rio Grande do Sul</td>\n      <td>R$\xa06.618.703,00</td>\n    </tr>\n    <tr>\n      <td>Goiás</td>\n      <td>R$\xa06.288.009,00</td>\n    </tr>\n    <tr>\n      <td>Bahia</td>\n      <td>R$\xa05.030.257,00</td>\n    </tr>\n    <tr>\n      <td>Santa Catarina</td>\n      <td>R$\xa04.971.700,00</td>\n    </tr>\n    <tr>\n      <td>Dist

In [9]:
tb_faturamento_produto_formatado = pd.DataFrame(tb_faturamento_produto['Faturamento'].apply(lambda x: format_currency(x, 'BRL', locale='pt_BR')))
tb_faturamento_produto_formatado = tb_faturamento_produto_formatado.reset_index()

# Formatação Html
tb_faturamento_produto_formatado = tb_faturamento_produto_formatado.to_html(index=False, justify='center', border=0).replace('<tbody>', '<tbody style="text_align: center; color: #484848; background: #F7F7F7">')


display(tb_faturamento_produto_formatado)

'<table class="dataframe">\n  <thead>\n    <tr style="text-align: center;">\n      <th>Produto</th>\n      <th>Faturamento</th>\n    </tr>\n  </thead>\n  <tbody style="text_align: center; color: #484848; background: #F7F7F7">\n    <tr>\n      <td>Notebook</td>\n      <td>R$\xa048.991.770,00</td>\n    </tr>\n    <tr>\n      <td>PC</td>\n      <td>R$\xa038.456.200,00</td>\n    </tr>\n    <tr>\n      <td>Monitor</td>\n      <td>R$\xa011.907.600,00</td>\n    </tr>\n    <tr>\n      <td>Impressora</td>\n      <td>R$\xa09.286.215,00</td>\n    </tr>\n    <tr>\n      <td>Tablet</td>\n      <td>R$\xa09.016.716,00</td>\n    </tr>\n    <tr>\n      <td>Nobreak</td>\n      <td>R$\xa08.669.062,00</td>\n    </tr>\n    <tr>\n      <td>Teclado</td>\n      <td>R$\xa01.193.580,00</td>\n    </tr>\n    <tr>\n      <td>Webcam</td>\n      <td>R$\xa0984.300,00</td>\n    </tr>\n    <tr>\n      <td>Mouse</td>\n      <td>R$\xa0725.872,00</td>\n    </tr>\n    <tr>\n      <td>Caixa de som</td>\n      <td>R$\xa0516.

In [10]:
tb_ticket_medio_formatado = pd.DataFrame(tb_ticket_medio['Ticket_medio'].apply(lambda x: format_currency(x, 'BRL', locale='pt_BR')))
tb_ticket_medio_formatado = tb_ticket_medio_formatado.reset_index()

# Formatação Html
tb_ticket_medio_formatado = tb_ticket_medio_formatado.to_html(index=False, justify='center', border=0).replace('<tbody>', '<tbody style="text_align: center; color: #484848; background: #F7F7F7">')


display(tb_ticket_medio_formatado)

'<table class="dataframe">\n  <thead>\n    <tr style="text-align: center;">\n      <th>Loja</th>\n      <th>Ticket_medio</th>\n    </tr>\n  </thead>\n  <tbody style="text_align: center; color: #484848; background: #F7F7F7">\n    <tr>\n      <td>Acre</td>\n      <td>R$\xa02.142,77</td>\n    </tr>\n    <tr>\n      <td>Sergipe</td>\n      <td>R$\xa01.735,33</td>\n    </tr>\n    <tr>\n      <td>Tocantins</td>\n      <td>R$\xa01.734,31</td>\n    </tr>\n    <tr>\n      <td>Rio Grande do Sul</td>\n      <td>R$\xa01.690,04</td>\n    </tr>\n    <tr>\n      <td>Pará</td>\n      <td>R$\xa01.683,75</td>\n    </tr>\n    <tr>\n      <td>Santa Catarina</td>\n      <td>R$\xa01.683,56</td>\n    </tr>\n    <tr>\n      <td>Pernambuco</td>\n      <td>R$\xa01.683,04</td>\n    </tr>\n    <tr>\n      <td>Paraná</td>\n      <td>R$\xa01.608,73</td>\n    </tr>\n    <tr>\n      <td>Bahia</td>\n      <td>R$\xa01.606,05</td>\n    </tr>\n    <tr>\n      <td>Mato Grosso do Sul</td>\n      <td>R$\xa01.600,95</td>\n  

In [11]:
tb_quantidade_produto_formatado = pd.DataFrame(tb_quantidade_produto['Quantidade'].apply(lambda x: "{:,}".format(x).replace(',', '.')))
tb_quantidade_produto_formatado = tb_quantidade_produto_formatado.reset_index()

# Formatação Html
tb_quantidade_produto_formatado = tb_quantidade_produto_formatado.to_html(index=False, justify='center', border=0).replace('<tbody>', '<tbody style="text_align: center; color: #484848; background: #F7F7F7">')


display(tb_quantidade_produto_formatado)

'<table class="dataframe">\n  <thead>\n    <tr style="text-align: center;">\n      <th>Produto</th>\n      <th>Quantidade</th>\n    </tr>\n  </thead>\n  <tbody style="text_align: center; color: #484848; background: #F7F7F7">\n    <tr>\n      <td>Impressora</td>\n      <td>13.285</td>\n    </tr>\n    <tr>\n      <td>Notebook</td>\n      <td>12.930</td>\n    </tr>\n    <tr>\n      <td>Monitor</td>\n      <td>9.923</td>\n    </tr>\n    <tr>\n      <td>Nobreak</td>\n      <td>6.778</td>\n    </tr>\n    <tr>\n      <td>Tablet</td>\n      <td>6.684</td>\n    </tr>\n    <tr>\n      <td>Teclado</td>\n      <td>6.631</td>\n    </tr>\n    <tr>\n      <td>Webcam</td>\n      <td>6.562</td>\n    </tr>\n    <tr>\n      <td>Caixa de som</td>\n      <td>6.535</td>\n    </tr>\n    <tr>\n      <td>PC</td>\n      <td>6.518</td>\n    </tr>\n    <tr>\n      <td>Mouse</td>\n      <td>6.481</td>\n    </tr>\n  </tbody>\n</table>'

In [12]:
!pip install -U kaleido

In [19]:
caminho_grafico = "/content/grafico.png"
grafico.write_image(caminho_grafico)

In [21]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Enviar um email para o setor responsável

corpo_email = f"""
<p>Prezados,</p>

<p>Segue o relatório de vendas do mês</p>

<p><b>Faturamento por loja:</b></p>
<p>{tb_faturamento_loja_formatado}</p>

<p><img src='cid:grafico'></p>

<p><b>Quantidade vendida por Produto:</b></p>
<p>{tb_faturamento_produto_formatado}</p>

<p><b>Faturamento por Produto:</b></p>
<p>{tb_quantidade_produto_formatado}</p>

<p><b>Ticket médio por loja:</b></p>
<p>{tb_ticket_medio_formatado}</p>


<p>Qualquer dúvida estou à disposição</p>

<p>Att.</p>

<p>Cleber Lopes</p>

"""

# Configurações do servidor SMTP
smtp_server = 'smtp.gmail.com'
smtp_port = 587
smtp_username = 'cleberlopestj2018@gmail.com'
smtp_password = 'kkffmlitvunnruvn'


# Configurações do e-mail
sender_email = 'cleberlopestj2018@gmail.com'
recipient_email = 'cleberlopestj@yahoo.com.br'
subject = 'Relatório de Vendas'
body = corpo_email

# Criar a mensagem
message = MIMEMultipart()
message['From'] = sender_email
message['To'] = recipient_email
message['Subject'] = subject

# Adicionar corpo da mensagem
message.attach(MIMEText(body, 'html'))

# Incorporando a imagem ao email
with open(caminho_grafico, "rb") as img_file:
    mime_img = MIMEImage(img_file.read(), name="grafico.png", _subtype="png")
    mime_img.add_header('Content-ID', '<grafico>')  # O '<>' é importante aqui
    mime_img.add_header('Content-Disposition', 'inline', filename="grafico.png")
    message.attach(mime_img)

# Conectar-se ao servidor SMTP e enviar e-mail
try:
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(smtp_username, smtp_password)
        server.sendmail(sender_email, recipient_email, message.as_string())
    print('E-mail enviado com sucesso!')
except Exception as e:
    print(f'Erro ao enviar e-mail: {e}')

E-mail enviado com sucesso!
